# Classification

In the previous [notebook](../2/End_to_end_machine_learning_project.ipynb), we worked on an end-to-end problem and used a housing dataset to predict a median house value. This was a regression problem as we were predicting a continous value. We used Linear Regression, Decision Tree, Random Forest and Support Vector Machine (We will cover all of these in next few chapters, the aim of previous notebook was to get sense of whole process).

This notebook is inspired by chapter 3 of the excellent book - [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) by Aurélien Géron. In this notebook, we will explore classification problems, where we predict which class does an instance belong to. We will MNIST dataset of 70,000 images of digits written by high-school students and employees of US Census Bureau. Each image is labelled with a digit it represents. `Scikit-Learn` let us download popular datasets. MNIST is one of them.


## Download the data.


In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

Each `Scikit-Learn` dataset will come in this standard dictionary structure. 
-  'data' represents the key containing an array with one row per instance and one column per feature.
- 'target' represents the key containing an array with labels.
- 'DESCR' represents the key for full description of the dataset.
-  'details' represents the key for more metadata from OpenML
For exploring more of these keys, go to [sklearn.datasets.fetch_openml](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_openml.html#sklearn.datasets.fetch_openml)

Let's get our *X* and *y*.

In [ ]:
X, y = mnist['data'], mnist['target']
print("X.shape :", X.shape)
print("y.shape :", y.shape)

Each image is actually a 28 x 28 pixel image. Therefore the data has 28 x 28 = 784 features. In other words, each pixel is represented as a feature. Let's take a look at one of the images. We will use `matplotlib`'s `imshow()` function.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import random

index = random.randint(0, 70000)
some_digit = X[index]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()


It looks like 1, let's see what label say.

In [ ]:
y[index]

It is indeed 1, the labels are string, so let's change them to actual numbers.

In [ ]:
import numpy as np

y = y.astype(np.uint8)

Let's plot some more digits

In [ ]:
def plot_digits_square(instances, images_per_row=5, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size, size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap=mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(4, 4))
example_images = X[-25:]
plot_digits_square(example_images, images_per_row=5)


## Train Test Split

Before continuing, let's do train-test split of the data. MNIST dataset is already split into training and test data. First 60,000 images are training and rest 10,000 images are test. The data is already shuffled for us.

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Training a Binary Classifier

For simplicity, let's first create a binary classifier, and try to identify only one digit for e.g. the number 5. Essentially we are build a 5-detector, which will classify the digits as either 5 or not 5.  The target vector will be 

In [ ]:
y_train_5 = (y_train == 5) #assigns true for all 5, false otherwise
y_test_5 = (y_test == 5)

Now, let's pick a classifier and train it. We will choose *Stochastic Gradient Descent (SGD)* as our first classifier using `SGDClassifier` of Scikit-Learn. This classifier has the advantage of handling very large dataset efficiently. One of the reasons for this is because it deals with one instance at a time. This also makes it suitable for online learning. The `SGDClassifier` relies on randomnness during training hence *Stochastic*.

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_classifier = SGDClassifier(random_state=42)
sgd_classifier.fit(X_train, y_train_5)

Let's see if how well it predicts. Let's first try for the `1` we had initially used.

In [ ]:
sgd_classifier.predict([some_digit])

It is indeed not a `5`. Let's evaluate the entire model's performance. 

## Performance Measures for Classification.

In this section, we will see how to measure performance of the Classification models. There are various nuances involved. Let's see if accuracy is a good measure or not.

### Measuring accuracy using cross-validation.

We usually use `cross_val_score()` to do off-the-shelf cross-validation. However, sometimes we want to have more control over this process. In that case we use `StratifiedKFold`. It performs stratified sampling to produce folds which contain a representative ratio of each class. At each iteration, we get the new test and train data, we clone the model, and train it fresh. After that we make prediction on test fold. This way, a model never sees the test data on which we predict.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

# We will have 3 folds
skfolds = StratifiedKFold(n_splits=3, shuffle=False)

for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_classifier)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print("Accuracy of the fold: ", n_correct/len(y_pred))

Wow, the performance looks good, *but is it?*. Imagine you have a dataset of emails which has either legitimate emails or spam emails. There are say 95% legitimate emails and 5% spam emails. Let's say you want to build a classifier to classify legitimate and spam emails. If for every prediction you say that email is legitimate, you will be right 95% of the time. 

Let's do the same with above case, let's just always predict that value is not 5. You can imagine what happens!

In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
from sklearn.model_selection import cross_val_score

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

As expected, we will always get around 90% accuracy! This is because we have 10% images as not 5. So if we always guess image is not 5, we will be right 90% of the time.


Clearly, the example above describe why accuracy is a bad performance measure for classifiers, especially when datasets are skewed. Let's try another performance measurement technique.

### Confusion Matrix

A much better way to evaluate performance of classification task is to look at *Confusion Matrix*. The ideas is to count the number of times instances of class A are classified as instances of Class B.

To compute confusion matrix, we need to have set of predictions so that those can be compared to the actual target. Let's use `cross_val_predict` to get those. Just like `cross_val_score` function, `cross_val_predict` performs K-fold cross-validation but instead of returning the evaluation scores, it returns the predictions made on each test fold.  This means that we get a clean prediction of each instance in the training set (clean because the model used for a certain prediction has never seen that prediction before, this is done by creating a new model for each set of folds, as we have done above using `StratifiedKFold`.

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_classifier, X_train, y_train_5, cv=3)

Now, let's get the confusion matrix using the `confusion_matrix` function.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

![alt tag](util/confusion_matrix.png)

<div class="alert alert-block alert-success">
Each row in a confusion matrix represents an <i>actual class</i> while each column represents a <i>predicted class </i>.  The first row of matrix represents the non-5 images (the <i>negative classs</i>). 53,892 of them were correctly classified as non-5 images (They are called <i>true negatives</i>), while the remaining 687 were wrongly classified as 5s (<i>false positives</i>). The second row represents the 5-image (the <i>positive class</i>).  The 1891 were classified as non-5s (<i>false negatives</i>) and while the remaining 3530 were correctly classified as 5s (<i>true positives</i>). A perfect classifier will only have true positives and true negatives, therefore it will *only* have non-zero values across the main diagonal.
</div>

<div class="alert alert-block alert-danger">
Remember! The convention to represent the various values may vary. The current convention is from what `Scikit-Learn` represents. In [wikipedia](https://en.wikipedia.org/wiki/Confusion_matrix) entry for confusion matrix, the true-positives are in first row, first column. Here we have it opposite.
</div>

`scikit-learn` also let's you plot the confusion matrix. Let's try that.

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(sgd_classifier, X_train, y_train_5, values_format="d", cmap=plt.cm.Blues)

We can also give some labels

In [ ]:
plot_confusion_matrix(sgd_classifier, X_train, y_train_5, 
                      values_format="d", cmap=plt.cm.Blues, display_labels=['not_5', '5'])

Let's assume we have the perfect classifier, in that case the confusion matrix will look like this.

In [ ]:
y_train_perfect_predictions = y_train_5

confusion_matrix(y_train_5, y_train_perfect_predictions)

The confusion matrix gives a good amount of detail, but sometimes we may prefer a more concise matrix. An interesting one is to look at the accuracy of the positive predictions, this is called ***precision*** of the classifier.

$$precision = \frac{TP}{TP+FP}$$

*TP* is true positives and *FP* are the number of false positives.

<div class="alert alert-block alert-info">
    Precision looks at the accuracy of of the positive predictions. What proportions of the positive indentifcations were actually correct? It looks at the vertical row of the confusion matrix.
</div>

A trivial way to get 100% perfect *precision* is to make only one prediction and get it right. (1+0/1 = 100%). However, that will not be very useful, since the classifier will ignore all but one positive instance. So *precision* is typically used along with *recall* also called *sensivity* or the *True Positive Rate (TPR)*. This is the ratio of positive instances that are correctly detected by the classifier.

$$recall = \frac{TP}{TP+FN}$$

*FN* is the number of false negatives. 

<div class="alert alert-block alert-info">
    Recall tells us what proportions of the actual positives was identified correctly. It looks at the horizontal row of the confusion matrix.
</div>

`Scikit-Learn` provides convinient ways to directly calculate precisiona and recall.

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Precision score:", precision_score(y_train_5, y_train_pred))
print("Racall score:", recall_score(y_train_5, y_train_pred))

So what this numbers tells us is that when our classifier classifies an image as 5, it is correct 83% of the time. Moreover, it is only detecting 65% of the 5 values.

This tension of precision and recall can be combined into a single metric *$F_1 score$* which takes into account both *precision* and *recall*. The *$F_1 score$* is the harmonic mean of precision and recall. Whereas the regular mean treats all values equally, the harmonic mean gives much more weight to low values. As a result, the classifier will only get a high F1 score if both recall and precision are high.

$$F_1 = \frac{2}{\frac{1}{precision} + \frac{1}{recall}} = 2 \times \frac{precision \times recall}{precision + recall} = \frac{TP}{TP + \frac{FN+FP}{2}}$$

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)

The $F_1 score$ favors the classifier which have similar precision and recall. However, it might not be what application wants. Let's take an example. Imagine we are building a youtube like service for the kids. We have a classifier which tells us whether the video being uploaded is safe for the kids or not. In this case, we definitely want a classfier that keeps safe videos (i.e. high precision) but it's okay to reject many good videos (i.e. low recall). It would be much better than a system with higher recall and low precision (where there is a chance that some videos might be bad). 

As an another example, if we are building a system for shoplifting in survellience images. It will be fine if your system has lower precision (say 30%) but as long as it has higher recalls (say 90%). Security guards might have false alerts but almost all shoplifters will be caught.

Unfortunately, we can't have it both ways. Increasing recall decreases precision and vice versa. This is called *precision/recall trade-off*.

## Precision Recall Trade off

For each new instance, our system computes a score based on *decision function*. If the score is higher than a *threshold*, the system assigns it othe positive class; otherwise, it is assigned to negative class. In the below image, there are 30 predictions made by an email classification model. Those to the right of the threshold are classified as *spam* and those to the left are classified as *not spam*. 

![alt tag](util/classification1.png)

Image source : [Google Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)

Let's first create the *confusion matrix* for it. 

|   |   |
|:---:|:---:|
|True Negatives (TN) 17 |  False Positives (FP) 2 |
|False Negatives (FN) 3   | True Positives (TP) 8 |

In this case, precision will be **percentage of emails we flagged as spam**  that were correctly classified.

$$Precision =  \frac{TP}{TP + FP} = \frac{8}{8+2} = 0.8$$

Recall will be the percentage of **actual spam emails** that were correctly classified

$$Recall =  \frac{TP}{TP + FN} = \frac{8}{8+3} = 0.73$$

We have 8 emails classified as spam, which are actually spam (TP = 8). We have 2 emails classified as spam but they are not spam (FP = 2). We have 17 emails classified as not spam, which are actually not spam (TN = 17). Finally we have emails which are falsely classified as not spam, which are actually spam (FN = 3). Therefore for precision we have identified 8 emails as spam correctly out of total 10 emails we classified as spam therefore 0.8. Similarly, out of total 11 spam emails, we identified 8 emails as spam, hence 0.73.

Now, let's move this classification threshold to right. 

![alt tag](util/classification2.png)

Let's see the *Confusion Matrix* for this case.


|   |   |
|:---:|:---:|
|True Negatives (TN) 18 |  False Positives (FP) 1 |
|False Negatives (FN) 4   | True Positives (TP) 7 |

In this case, the number of false positives decreases, but we also see rise in false negative. Therefore, precision increases but recall decreases.

Remember, precision is the **percentage of emails we flagged as spam**  that were correctly classified.

$$Precision =  \frac{TP}{TP + FP} = \frac{7}{7+1} = 0.88$$

Recall is the percentage of **actual spam emails** that were correctly classified

$$Recall =  \frac{TP}{TP + FN} = \frac{7}{7+4} = 0.64$$

In the last case, let's move classification threshold to the left, it will result in decrease of precision and increase of recall.

![alt tag](util/classification3.png)


|   |   |
|:---:|:---:|
|True Negatives (TN) 16 |  False Positives (FP) 3 |
|False Negatives (FN) 2   | True Positives (TP) 9 |

$$Precision =  \frac{TP}{TP + FP} = \frac{9}{9+3} = 0.75$$

$$Recall =  \frac{TP}{TP + FN} = \frac{9}{9+2} = 0.82$$

`scikit-learn` doesn't let you set the threshold directly, but we get access to decision scores, which we can use to make prediction. This can be done using `decision_function` instead of `predict` method. `decision_function` will give you score for each instance, and then we can use any threshold to make predictions. Let's see this in code.

In [ ]:
y_scores = sgd_classifier.decision_function([some_digit])
print(y_scores)

threshold = 0
y_some_digit_pred = (y_scores > threshold)
print(y_some_digit_pred)

`SGDClassifier` by default uses 0 as threshold, therefore, in this case, result is same as predict function i.e. False.  We can change the threshold to any value and we will see the precision-recall tradeoff.

How do we decide what is the good value for threshold. This can be done by using `decision_function` as method in `cross_val_predict`. Let's try that.

In [ ]:
y_scores = cross_val_predict(sgd_classifier, X_train, y_train_5, cv=3, method="decision_function")

We just calculated scores for each instance. Now we can use these scores in `precision_recall_curve` to compute precision and recall for all possible threshold. Let's do that.  


In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

Lastly we can use `matplotlib` to plot precision and recall as function of various threshold values. 

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds, title):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g--", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16)
    plt.xlabel("Thresholds", fontsize=16)
    plt.title(title)
    plt.axis([-50000, 50000, 0, 1])


plt.figure(figsize=(8,4))
plt.grid(True)
plot_precision_recall_vs_threshold(precisions, recalls, thresholds, 
                                   "Precision, Recall for various thresholds for SGD classifier")

<div class="alert alert-block alert-warning">
When threshold is increased precision curve looks bumpier, because precision may sometimes go down when threshold is raised, although it usually goes up. On the other hand, Recall will always decrease or likely stays the same. The number of true positives may decrease or stay the same and will the cause the number of false negatives to increase or stay the same. 
</div>


Another way to understand the precision-recall tradeoff is to plot precision directly against recall.

In [ ]:
def plot_precision_vs_recall(precisions, recalls, title):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.title(title)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8,4))
plot_precision_vs_recall(precisions, recalls, "Precision vs Recall for SGD Classifier")

We can see that precision starts falling around 75% recall.  It is usually there we want to select a precision/recall trade-off. However ofcourse, as discussed above choice depends on project requirements. Using these two plots we can decide what threshold we need to choose. For e.g. let's say we go with 90% precision, in that case, the first plot will tell us the right threshold to use. In this case it is ~4000. Let's see how we use that in code.

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
threshold_90_precision

Now, we can modify our labels this way.

In [ ]:
y_train_pred_90 = (y_scores >= threshold_90_precision)
print("Precision: ", precision_score(y_train_5, y_train_pred_90))
print("Recall: ", recall_score(y_train_5, y_train_pred_90))

## The ROC curve

The *Receiver Operating Characteristic (ROC)* curve is another metric to measure the performance of binary classifier. It is a graph which shows the performance of classification model at all classification thresholds. The curve plots two parameters.

- True Positive Rate (TPR) which is synonym for recall.
$$TPR = \frac{TP}{TP+FN}$$

- False Positive Rate (FPR). It is the ratio of negative instances that were incorrectly classified as positive. It is 1 - the *True Negative Rate(TNR)* which is ratio of negative instances that were correctly classified as negative. The TNR is also called *Specificity*. Hence the ROC curve plots *sensitivity* (recall) vs 1 - *specificity*.  
$$TPR = \frac{FP}{FP+TN}$$

In other words, an ROC curve plots TPR vs FPR at various classification thresholds. Lowering the classification threshold classifies more items as positive, thus increasing both false positives and true positives. 

`Scikit-Learn` provides `roc_curve` to compute TPR, FPR at various threshold values.


In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

Now, let's plot the curve. 

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])                              
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) 
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    
    plt.grid(True)                                            
    

plot_roc_curve(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.show()

We can see that the higher the recall (TPR), the more false positives (FPR) the classifier produces. The straight dotted line represents when we are just randomly assigining a class. The model is better if the curve line representing the model is farther from this straight dotted line.

### Area under the ROC curve (AUC)

AUC stands for "Area under the ROC curve". AUC measures the entire two-dimensional area underneath the ROC curve. If we integrated curve of FPR vs TPR at various thresholds ranging from (0,0) to (1,1), we will get the area under this curve.
AUC lets us know the aggregate performance of the model across all possible classification threshold.One way of interpreting AUC is as the probability that the model ranks a random positive example more highly than a random negative example. The perfect classifier will have area under the curve as 1, while a purely random classifier will have 0.5.

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_scores)

<div class="alert alert-block alert-warning">
This looks great but we need to remember our positive class (5) is rare. As a rule of thumb, when positive class is rare or we care a lot about false positive than the false negative, we should prefer Precision-Recall curve over ROC curve. The ROC curve and AUC here gives us that model is really good, this is mostly because there are few positives (5s) compared to the negatives (non-5s).Looking at Precision-Recall curve, we can see that there is a room for improvement. The curve should be closer to top-right corner.
 </div>
 
Let's try a hopefully better classifier `RandomForestClassifier`. We will go in detail in later chapter on how this works, here we will directly use and how it does on the various metrics we explored. Also to note that `RandomForestClassifier` doesn't have `decision_function` instead it has `predict_proba` function. `Scikit-Learn` classifiers usually have one or the other or both. The `predict_proba` gives an array containing a row per instance and column per class, each containing probability that the given instance belongs to a given class.  

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, method='predict_proba')

let's explore `y_probas_forest`.

In [ ]:
print(y_probas_forest[0], y_train_5[0])

This basically tells us that first instance has 0.89 probability of being a 5, and 0.11 probability of not being a 5. Since `roc_curve` takes the labels and scores, but instead of scores we can give it positive class's probability.

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # second column represents positive class's proba
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

Let's plot the ROC curve of this `RandomForestClassifier` along with ROC curve of `SGDClassifier` we had before. This will let us compare the two models easily.

In [ ]:
plt.plot(fpr, tpr, "b:", label="SGD")
plt.plot(fpr_forest, tpr_forest, label="Random Forest")
plt.plot([0, 1], [0, 1], 'k--')
plt.legend(loc="lower right")
plt.grid(True)
plt.figure(figsize=(8, 6))
plt.show()

We can clearly see, the Random forest model is way better. Let's also calculate AUC

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)

Now, we are talking!! It is much better AUC. Let's also see precision and recall for this model and compare it with SGD

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions_rf, recalls_rf, thresholds_rf = precision_recall_curve(y_train_5, y_scores_forest)

In [ ]:
plt.plot(recalls, precisions, "b-", linewidth=2, label="SGD")
plt.plot(recalls_rf, precisions_rf, linewidth=2, label="Random Forest")
plt.xlabel("Recall", fontsize=16)
plt.ylabel("Precision", fontsize=16)
plt.title("Precision Recall curve for the two classifiers")
plt.legend(loc="lower left")
plt.axis([0, 1, 0, 1])
plt.grid(True)
plt.figure(figsize=(8,4))

RandomForest is much farther to right corner. It has high precision and high recall than SGD classifier. Let's calculate recall at 0.99 precision

In [ ]:
threshold_99_precision = thresholds_rf[np.argmax(precisions_rf >= 0.99)]
threshold_99_precision

In [ ]:
y_train_pred_99 = (y_scores_forest >= threshold_99_precision)
print("Precision: ", precision_score(y_train_5, y_train_pred_99))
print("Recall: ", recall_score(y_train_5, y_train_pred_99))

At 99% precision, we have 86% recall! This is a better model in every way!

## MultiClass Classification

So far we have seen classifiers for two classes (5 or not 5) called binary classifier, now let's focus on classifiers which can distinguish or classify more than 2 classes.  The problem we started with of classifying MNIST images from numbers 0-9 is a multiclass classification problem, here number of classes are 10. 

Some algorithms e.g. Logistic Regression, Random Forest Classifiers, and Naive Bayes Classifiers are capable of handling multiple classes natively. Others such as Stochastic Gradient Descent (SGD, we just saw) and Support Vector Machines are strictly binary classifiers. 

There are some strategies where we can perform multiclass classification using multiple binary classification.

*One-versus-the-rest (OvR)* or *One-versus-All* classifier: In this approach, we create a classifier for each class (just like we created a classifier for just 5, i.e. just like 5-detector, we can build 0-detector, 1-detector, 2-detector and so on..). Then when we want to classify a new instance say a new image of a number, we get the decision score from each classifier and select the classifier which has the highest score. 

*One-versus-One (OvO)* Classifier: In this approach, we build a classifier for each pair of classes. e.g. we build one classifier to distinguish between 0s and 1s, then we build one for 1s and 2s, then 2s and 3s and so on. If there are *N* classes we will have *N x (N-1) / 2* classifiers. For our MNIST problem, we will have to train 45 binary classifiers. For a new instance of the image, we will have to run through 45 classifiers, and then see which class wins the most duels.


Algorithms like Support Vector Machines scale very poorly with size of the dataset. For such algorithms, OvO is preferred approach as it is faster to train many classifiers on small training sets than to train few classifiers on large training set. For most binary classification algorithms, we prefer OvR.

`Scikit-Learn` lets you us a binary classification algorithm for a multiclass classification task and then it automatically runs OvR or OvO, depending on algorithm. Let's build a Support Vector Machine classifier (which again we will cover later)



In [ ]:
from sklearn.svm import SVC

svm_clf = SVC()
svm_clf.fit(X_train, y_train) #all classes not just 5
svm_clf.predict([some_digit])


Our `some_digit` is indeed 5. Let's see what scores did we have for this instance for all the classes.

In [ ]:
some_digit_scores = svm_clf.decision_function([some_digit])
some_digit_scores

In [ ]:
svm_clf.classes_

In [ ]:
svm_clf.classes_[np.argmax(some_digit_scores)]

We can map them to individual classes by using `classes_` attribute. Remember that order of attributes in `classes_` may not always be ordered. Although In this case, it is ordered. See 5 wins here.We can also choose the strategy on our own by using `OneVsOneClassifier` or `OneVsRestClassifier`. For example we can use the same 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, y_train)
ovr_clf.predict([some_digit])


Similarly, we can train an `SGDClassifier` 

In [ ]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)
sgd_clf.predict([some_digit])

`Scikit-Learn` will use OvR in this case. We can verify that using `decision_function` and see the number of outputs.

In [ ]:
sgd_clf.decision_function([some_digit])

We can see a really good score for the 5, and rest are negative. Let's see how we can evaluate it using `cross_val_score`

In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

We are getting around 86% accuracy over all folds, considering a random classifier will give you accuracy around 10%, this is a good score. Let's try to improve by scaling the inputs. We will use `StandardScaler`.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

Ok, our accuracy improved to 89% from 86%. Neat!

## Error analysis.

In our machine learning checklist in previous notebook, One of the steps after choosing your model is to understand where it makes the error and improve it.

Let's see where our classifier is making mistakes and see what we can do about it. We will use `cross_val_predict` and `confusion_matrix` like we did earlier.

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)
cm = confusion_matrix(y_train, y_train_pred)
cm

In [ ]:
def plot_confusion_matrix_local(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)
    
plot_confusion_matrix_local(cm)

Most of the images are in main diagonal, as it is showing greener, that means our classifier is doing good. Now, let's try analyze the errors. First we divide each value in confusion matrix by the number of images in the corresponding class so that you can compare error rates instead of the absolute number of errors. (Otherwise, the classes with more instances will look really bad. 

In [ ]:
row_sums = cm.sum(axis=1, keepdims=True)
norm_cm = cm / row_sums

Let's also fill the diagonals with 0s and then plot the confusion matrix again.

In [ ]:
np.fill_diagonal(norm_cm, 0)
plot_confusion_matrix_local(norm_cm)

Remember that rows represents the actual classes, while column represents the predicted classes. The column for class 8 is quite bright, that means many images are wrongly classified as 8. However the row of 8 looks okay, that means 8s usually get classified as 8. 3 and 5 are often get confused in both the directions.

Now based on this data, you can probably think of ways to improve our results. Since may be we should spend more efforts on reducing the false 8s. We can do this by gaining more data on images which gets classified wrongly as 8. Another way to do it is to engineer new features that would help the classifier, for e.g. 8 has 2 loops, 6 has 1 and 5 has none. We can preprocess the images in such a way that closed loops stand out more.

Let's plot 3s and 5s and see why they classified as each other.

In [ ]:
cl_a = 3
cl_b = 5

X_aa = X_train[(y_train == cl_a) & (y_train_pred == cl_a)]
X_ab = X_train[(y_train == cl_a) & (y_train_pred == cl_b)]
X_ba = X_train[(y_train == cl_b) & (y_train_pred == cl_a)]
X_bb = X_train[(y_train == cl_b) & (y_train_pred == cl_b)]

plt.figure(figsize=(8, 8))
plt.subplot(221)
plot_digits_square(X_aa[:25], images_per_row=5)
plt.subplot(222)
plot_digits_square(X_ab[:25], images_per_row=5)
plt.subplot(223)
plot_digits_square(X_ba[:25], images_per_row=5)
plt.subplot(224)
plot_digits_square(X_bb[:25], images_per_row=5)

Now, two 5 x 5 blocks on the left show digits classified as 3s , while the two 5x5 block right show images classified as 5
Some of the digits that the classifier gets wrong are very badly written that even a human would have trouble classifying them (e.g., In the lower right block, the 5 in the first row and second column truly looks like a badly written 3). However, most misclassified images seem like obvious errors to us, and it’s hard to understand why the classifier made the mistakes it did.3 The reason is that we used a simple `SGDClassifier`, which is a linear model. All it does is assign a weight per class to each pixel, and when it sees a new image it just sums up the weighted pixel intensities to get a score for each class. So since 3s and 5s differ only by a few pixels, this model will easily confuse them.

So what do we do in this case? We see that the how top line is joined to the bottom arc in 3 and 5. In 5, the joining happens on the left, on the other hand in 3, the joining happens on the right. If 3's line is drawn slightly shifted to the left, the classifier will classify this as 5, and vice versa. Therefore, we can reduce the 3/5 confusion by preprocessing to center the images well and ensure that they are not too rotated. This will reduce other errors too.

Extra: We can also use `plot_confusion_matrix` by the `scikit-learn` to do this. Let's just draw it for reference in future

In [ ]:
plt.figure(figsize=(10,10))
plot_confusion_matrix(sgd_clf, X_train, y_train, 
                      values_format="d", cmap=plt.cm.Blues, ax=plt.gca())

Notice that we are using `X_train` and not `X_train_scaled`. Also the above confusion matrix is with cross validation and this one is not. Therefore, the results might not be same as above. 

## Multilabel classification

Another type of classification problem is when we want the classifier to output multiple classes for each instance. Imagine a case of movie genre classification, a movie can be both thriller and crime, or it can be drama as well as romance. If we train a classifier like that on multiple labels, we will have an output like *\[1, 0, 1, 0\]* where first 1 represents drama genre, the second 0 represents absence of thriller, while third 1 represents presence of romance, and last 0 represents absence of crime. Such a classification system that outputs multiple binary tags is called *multilabel classification* system.

Let's extend our current problem to multilabel classification. The below code creates a multi-label `y_multilabel` array containing two target labels for each digit image: The first indicates whether the digit is large i.e. 7, 8 or 9, and the second represents if the digit is odd or not. We create a `KNeighborsClassifier` instance which supports multi-label classification, and thus we train it with multiple targets array.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]
knn_clf = KNeighborsClassifier()

knn_clf.fit(X_train, y_multilabel)


Now, let's make a prediction. Remember our `some_digit` is 5.

In [ ]:
knn_clf.predict([some_digit])

It seems we got it right, the digit 5 is indeed less than 7 and it is odd. 

There are multiple ways to evaluate a multilabel classifier and as we know right metrics depend on the project requirement. One approach is to measure $F_1$ score for each individual label (or any other binary classifier metric) and then average the score across all the labels.

In [ ]:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)

In [ ]:
f1_score(y_multilabel, y_train_knn_pred, average="macro")

We are assuming here that the labels have equal importance, but that might not be the case in your project. We can assign each label a weight and set `average="weighted"` in the preceding code.


## Multioutput Classification

Another type of classification task is *multioutput-multiclass classification* or simply *multioutput classification*. It is simply generalization of *multilabel classification* we just saw where we assumed that each label represents a binary values, instead in this case each label can be multiclass i.e. it can have more than 2 possible classes.

Author has given an example to illustrate this. Let's build a system that removes noise from images. It will take as input a noisy digital image and will hopefully output a clean digit image, represented as an array of pixel intensitities, just like the MNIST images. Notice that the classifier's output will be multilabel (one label per pixel) and each label can have multiple values (pixel intensity ranges from 0 to 255). It is therefore a multioutput classification system. 

Let's start by creating a training and test set by adding some random noise to images.

In [ ]:
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise

y_train_mod = X_train
y_test_mod = X_test

Let's peek at an input and output sample

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
some_index = 0
plt.subplot(121)
plot_digit(X_test_mod[some_index])
plt.subplot(122)
plot_digit(y_test_mod[some_index])

So here it is, we have noisy input image and clean target image. Now let's train the classifier.




In [ ]:
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)

Voila! We did get a clean digit back!

## Exercises

1. Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the `KNeighborsClassifier` works quite well for this task; you just need to find good hyperparameter values (try a grid search on the `weights` and `n_neighbors` hyperparameters).

**Solution** In this case, as we did before, we will do a grid search with weights as uniform and as distance, and we will try 3, 4, 5 neighbors. We will also use -1 for `n_jobs` so search can run in parallel.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "weights" : ["uniform", "distance"],
        "n_neighbors" : [3, 4, 5]
    }
]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv = 3, verbose=3, n_jobs=16)
grid_search.fit(X_train, y_train)